### Recommendation Systems

Recommendation Systems is a system that suggests the items to the users baed on their preferences.

Examples:

1.   Netflix recommending movies.

2.   Amazon suggesting products.

3.   Spotify suggesting songs.

#### Collaborative Filtering


Collaborative Filtering is one of the most common techniques for recommendation systems.


### Steps used in this Algorithm:-

1.  Import all the necessary libraries

2.  Create the DataFrame

3.  Create the user-item matrix

4.  Fill missing values with 0

5.  Compute the similarity between the users using cosine similarity

6.  Recommend the Items for the users

7.  Provide the top recommendations for the users

### Step 1: Import all the necessary libraries

In [321]:
import  numpy             as  np
import  pandas            as  pd
import  matplotlib.pyplot as  plt
import  seaborn           as  sns

### Step 2: Create the DataFrame

In [322]:
data = {
    'user': ['Alice', 'Alice', 'Bob', 'Bob', 'Carol', 'Carol', 'Dave', 'Eve'],
    'item': ['Matrix', 'Titanic', 'Matrix', 'Avengers', 'Titanic', 'Avengers', 'Titanic', 'Avengers'],
    'rating': [5, 3, 4, 5, 2, 5, 4, 4]
}

In [323]:
df = pd.DataFrame(data)

In [324]:
df

,user,item,rating
0,Alice,Matrix,5
1,Alice,Titanic,3
2,Bob,Matrix,4
3,Bob,Avengers,5
4,Carol,Titanic,2
5,Carol,Avengers,5
6,Dave,Titanic,4
7,Eve,Avengers,4


### OBSERVATIONS:

1. The above dataset contains the list of all the users who have rated their particular items.

### Step 3: Create the user-item matrix

In [325]:
user_item_matrix = df.pivot(
    index      = 'user'          ,
    columns    = 'item'          ,
    values     = 'rating'
)

In [326]:
user_item_matrix

item,Avengers,Matrix,Titanic
user,,,
Alice,NaN,5.0,3.0
Bob,5.0,4.0,NaN
Carol,5.0,NaN,2.0
Dave,NaN,NaN,4.0
Eve,4.0,NaN,NaN


### Step 4: Fill missing values with 0

In [327]:
user_item_matrix = user_item_matrix.fillna(0)

In [328]:
user_item_matrix

item,Avengers,Matrix,Titanic
user,,,
Alice,0.0,5.0,3.0
Bob,5.0,4.0,0.0
Carol,5.0,0.0,2.0
Dave,0.0,0.0,4.0
Eve,4.0,0.0,0.0


### Step 5: Compute the similarity between the users using cosine similarity

In [329]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)

In [330]:
user_similarity

array([[1.        , 0.53567158, 0.19107893, 0.51449576, 0.        ],
       [0.53567158, 1.        , 0.72501849, 0.        , 0.78086881],
       [0.19107893, 0.72501849, 1.        , 0.37139068, 0.92847669],
       [0.51449576, 0.        , 0.37139068, 1.        , 0.        ],
       [0.        , 0.78086881, 0.92847669, 0.        , 1.        ]])

In [331]:
user_item_matrix

item,Avengers,Matrix,Titanic
user,,,
Alice,0.0,5.0,3.0
Bob,5.0,4.0,0.0
Carol,5.0,0.0,2.0
Dave,0.0,0.0,4.0
Eve,4.0,0.0,0.0


In [332]:
user_item_matrix.index

Index(['Alice', 'Bob', 'Carol', 'Dave', 'Eve'], dtype='object', name='user')

In [333]:
user_item_matrix.columns

Index(['Avengers', 'Matrix', 'Titanic'], dtype='object', name='item')

In [334]:
### Construct the DataFrame from the above data

df = pd.DataFrame(data = user_similarity, index = user_item_matrix.index,columns = user_item_matrix.index)

In [335]:
df

user,Alice,Bob,Carol,Dave,Eve
user,,,,,
Alice,1.000000,0.535672,0.191079,0.514496,0.000000
Bob,0.535672,1.000000,0.725018,0.000000,0.780869
Carol,0.191079,0.725018,1.000000,0.371391,0.928477
Dave,0.514496,0.000000,0.371391,1.000000,0.000000
Eve,0.000000,0.780869,0.928477,0.000000,1.000000


### Step 6: Recommend the Items for the users

In [336]:
def recommend_item(user, user_item_matrix,user_similarity_df, top_n=2):
    similar_users = user_similarity_df[user].sort_values(ascending=False)[1:]  # exclude self
    recommendations = pd.Series(dtype=float)
    
    for u, sim in similar_users.items():
        ratings = user_item_matrix.loc[u]
        recommendations = recommendations.add(ratings * sim, fill_value=0)


    # Remove items already rated by user
    recommendations = recommendations.drop(user_item_matrix.loc[user][user_item_matrix.loc[user] > 0].index)
    recommendations = recommendations.sort_values(ascending=False)
    return recommendations.head(top_n)

### Step 7: Provide the top recommendations for the users

In [337]:
top_recommendations = recommend_item('Alice', user_item_matrix, df)
print("\nTop Recommendations for Alice:")
print(top_recommendations)


Top Recommendations for Alice:
item
Avengers    3.633753
dtype: float64
